In [1]:
# Import packages
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import seaborn as sns
import re
import string
from ignore_SSL_errors import no_ssl_verification
import concurrent.futures

In [2]:
# Import list of euronext stocks => https://live.euronext.com/en/products/equities/list
file_name = "Euronext_Equities_2022-01-09.csv"
stock_detail = pd.read_csv(file_name, sep=';')
stock_detail.head()

,Name,ISIN,Symbol,Market,Trading Currency,Open,High,Low,Last,Last Date/Time,Time Zone,Volume,Turnover
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,22.40,22.40,22.40,22.40,07/01/22 15:11,CET,5,112.00
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK,116.00,117.00,114.50,115.50,07/01/22 16:25,CET,113866,13185546.00
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,4.94,4.94,4.88,4.92,07/01/22 17:35,CET,3296,16176.965
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,9.80,9.80,9.80,9.80,07/01/22 17:06,CET,10608,103958.40
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,0.18,0.18,0.18,0.18,16/12/21 17:04,CET,58571,10542.78


In [3]:
# Get distinct stock details
stock_detail['Market'].unique()

array(['Euronext Growth Paris', 'Oslo B�rs', 'Euronext Paris',
       'Euronext Expand Oslo', 'Euronext Access Paris',
       'Euronext Amsterdam', 'Euronext Growth Oslo', 'Euronext Brussels',
       'Euronext Expert Market', 'Euronext Brussels, Paris',
       'Euronext Paris, Amsterdam', 'Euronext Paris, Brussels',
       'Euronext Brussels, Amsterdam', 'Euronext Access Lisbon',
       'Euronext Amsterdam, Brussels', 'Euronext Dublin',
       'Euronext Lisbon', 'Euronext Amsterdam, Brussels, Paris',
       'Euronext Amsterdam, Paris', 'Traded not listed Brussels',
       'Euronext Growth Paris, Brussels', 'Euronext Growth Brussels',
       'Euronext Access Brussels', 'Euronext Growth Dublin',
       'Euronext Paris, Amsterdam, Brussels', 'Euronext Growth Lisbon'],
      dtype=object)

In [4]:
# Create a mapping for the exchange cities to yahoo codes
mapping_exchanges = {
    'Amsterdam':'AS',
    'Brussels':'BR',
    'Dublin':'IR',
    'Lisbon':'LS',
    'Oslo':'OL',
    'Paris':'PA' 
    }


In [5]:
# Create a unique list of cities
cities_list = list(mapping_exchanges.keys())
cities_list

['Amsterdam', 'Brussels', 'Dublin', 'Lisbon', 'Oslo', 'Paris']

In [6]:

# Function that takes in a string (input_text) and a list of different cities
# Returns the first city mentioned in the string that is in the cities list
from typing import List

def return_first_city(input_text: str, cities_list: List[str]) -> str:
    split_word_list = re.sub('['+string.punctuation+']', '', input_text).split()
    for word in split_word_list:
      if word in cities_list:
        result = word
        break
      else:
        result = "No city mentioned"
    return result


assert return_first_city('Amsterdam, Paris, Brussels', [
                         'Amsterdam', 'Brussels', 'Dublin', 'Lisbon', 'Oslo', 'Paris']) == 'Amsterdam'


In [7]:
# Create column to get the main city
stock_detail['main_city'] = stock_detail['Market'].apply(
    lambda x: return_first_city(x, cities_list))
stock_detail.head()

,Name,ISIN,Symbol,Market,Trading Currency,Open,High,Low,Last,Last Date/Time,Time Zone,Volume,Turnover,main_city
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,22.40,22.40,22.40,22.40,07/01/22 15:11,CET,5,112.00,Paris
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK,116.00,117.00,114.50,115.50,07/01/22 16:25,CET,113866,13185546.00,Oslo
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,4.94,4.94,4.88,4.92,07/01/22 17:35,CET,3296,16176.965,Paris
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,9.80,9.80,9.80,9.80,07/01/22 17:06,CET,10608,103958.40,Paris
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,0.18,0.18,0.18,0.18,16/12/21 17:04,CET,58571,10542.78,Paris


In [8]:
# Get the Yahoo city mapping
stock_detail['yahoo_city_code'] = stock_detail['main_city'].map(mapping_exchanges)
stock_detail.head()

,Name,ISIN,Symbol,Market,Trading Currency,Open,High,Low,Last,Last Date/Time,Time Zone,Volume,Turnover,main_city,yahoo_city_code
0,1000MERCIS,FR0010285965,ALMIL,Euronext Growth Paris,EUR,22.40,22.40,22.40,22.40,07/01/22 15:11,CET,5,112.00,Paris,PA
1,2020 BULKERS,BMG9156K1018,2020,Oslo B�rs,NOK,116.00,117.00,114.50,115.50,07/01/22 16:25,CET,113866,13185546.00,Oslo,OL
2,2CRSI,FR0013341781,2CRSI,Euronext Paris,EUR,4.94,4.94,4.88,4.92,07/01/22 17:35,CET,3296,16176.965,Paris,PA
3,2MX ORGANIC,FR0014000T90,2MX,Euronext Paris,EUR,9.80,9.80,9.80,9.80,07/01/22 17:06,CET,10608,103958.40,Paris,PA
4,2MX ORGANIC BS,FR0014000TB2,2MXBS,Euronext Paris,EUR,0.18,0.18,0.18,0.18,16/12/21 17:04,CET,58571,10542.78,Paris,PA


In [9]:
# Create the full city code
stock_detail['yahoo_code'] = stock_detail['Symbol'].str.cat(
    stock_detail['yahoo_city_code'], sep='.')

In [10]:
# Filter to get the fields needed and save to a csv
final_df = stock_detail[["Name", "ISIN", "main_city", "yahoo_code"]]
final_df.to_csv('data_stock_info.csv', sep='|', index=False)

In [11]:
# Create a function that extracts price data from yahoo using a ticker code
def get_yahoo_data(yahoo_code):
    price_history = None
    try:
        stock = yf.Ticker(yahoo_code)
        price_history = stock.history(period="5y", interval="1d")
    except Exception:
        pass
    return (yahoo_code, price_history)


final_price_history_list = []

# Run through all the ticker codes and extract price data
with no_ssl_verification():
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(
            get_yahoo_data, yahoo_code): yahoo_code for yahoo_code in final_df['yahoo_code']}
        for future in concurrent.futures.as_completed(futures):
            yahoo_code, price_history = future.result()
            print(f'Extracting info for {yahoo_code}')
            # If the result is a data frame, add come additional fields and add it to a list
            if isinstance(price_history, pd.DataFrame):
                price_history['yahoo_code'] = yahoo_code
                price_history['pct_change'] = price_history['Close'].pct_change()
                price_history['log_ret'] = np.log(
                    price_history['Close']) - np.log(price_history['Close'].shift(1))
                final_price_history_list.append(price_history)
                rows = len(price_history.index)
                print(f"{rows} acquired")

# Combine all the final results and save to csv
final_price_history_data = pd.concat(final_price_history_list)
final_price_history_data.to_csv('data_price_history.csv', sep='|')


- MLATV.PA: No data found, symbol may be delisted
Extracting info for MLATV.PA
0 acquired
Extracting info for nan
Extracting info for 2MXBS.PA
1 acquired
Extracting info for 2MX.PA
280 acquired
Extracting info for AASB.OL
1253 acquired
Extracting info for AALB.AS
1280 acquired
Extracting info for 2CRSI.PA
912 acquired
Extracting info for 2020.OL
629 acquired
Extracting info for AB.PA
1281 acquired
Extracting info for ABI.BR
1280 acquired
Extracting info for ALMIL.PA
1281 acquired
Extracting info for 5PG.OL
1254 acquired
Extracting info for ASP.PA
1281 acquired
Extracting info for ABCA.PA
1281 acquired
Extracting info for ABNX.PA
1281 acquired
Extracting info for ABG.OL
1254 acquired
Extracting info for ABEO.PA
1281 acquired
Extracting info for AAC.PA
159 acquired
Extracting info for ACAN.PA
1281 acquired
Extracting info for ABVX.PA
1281 acquired
Extracting info for ACPH.BR
987 acquired
Extracting info for ABN.AS
1280 acquired
Extracting info for ACCEL.AS
1280 acquired
Extracting info f

C:\Users\Michael.CamdenSmith\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Extracting info for BFIT.AS
1280 acquired
Extracting info for BLV.PA
153 acquired
Extracting info for BESI.AS
1280 acquired
Extracting info for BEAB.BR
1279 acquired
Extracting info for BEFB.BR
1280 acquired
Extracting info for FBEL.PA
1281 acquired
Extracting info for BEKB.BR
1280 acquired
Extracting info for nan
Extracting info for nan
Extracting info for BELCO.OL
1254 acquired
Extracting info for SLBEN.LS
1280 acquired
Extracting info for nan
Extracting info for BELR.BR
1279 acquired
Extracting info for BCS.OL
186 acquired
Extracting info for BEN.PA
1281 acquired
Extracting info for BGBIO.OL
1191 acquired
Extracting info for ALDBL.PA
1280 acquired
Extracting info for BEWI.OL
344 acquired
Extracting info for BELU.BR
1279 acquired
Extracting info for BBED.AS
1280 acquired
Extracting info for BEVER.AS
1280 acquired
Extracting info for ALTUV.PA
900 acquired
Extracting info for nan
Extracting info for BB.PA
1281 acquired
Extracting info for ALBLD.PA
1281 acquired
Extracting info for BFIS